### Estimación de sexo de propietarios de viviendas

Para este cálculo, se considera el subconjunto de viviendas que son propias o propias en proceso de pago. Para la identificación del dueño, se utiliza únicamente el del primer dueño que captura la ENIGH.

In [1]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr,gt)
font_add_google("Poppins", "pop")
showtext_auto()

Cargando paquete requerido: pacman



In [2]:
#Crear directorio para guardar los archivos
dir.create("data", showWarnings = FALSE)

In [3]:
#Datos básicos de URL para descarga
url_basica <- "https://www.inegi.org.mx/contenidos/programas/enigh/nc/"
segunda_parte <- "microdatos/enigh"
years <- c("2016", "2018", "2020", "2022","2024")
tipos <- c("ns_viviendas", "ns_poblacion")

In [4]:
#Descarga archivos
for (i in seq_along(years)) {
  for (tipo in tipos) {
    fin <- paste0("_", tipo, "_csv.zip")
    url <- paste0(url_basica, years[i], "/", segunda_parte, years[i], fin)
    destfile <- paste0("data/", years[i], fin)
    
    # Descargar archivo zip
    download.file(url, destfile = destfile)

    # Crear carpeta temporal de extracción (por año y tipo)
    carpeta_temporal <- paste0("data/tmp_", tipo, "_", years[i])
    dir.create(carpeta_temporal, showWarnings = FALSE)
    
    # Descomprimir en la carpeta temporal
    unzip(destfile, exdir = carpeta_temporal)

    # Detectar archivo .csv en la carpeta temporal
    archivos_csv <- list.files(carpeta_temporal, pattern = "\\.csv$", full.names = TRUE)
    
    if (length(archivos_csv) == 1) {
      # Eliminar prefijo "ns_" si existe
      tipo_limpio <- sub("^ns_", "", tipo)
      nombre_destino <- paste0("data/", tipo_limpio, "_", years[i], ".csv")
      file.rename(archivos_csv, nombre_destino)
    } else {
      warning(paste("No se encontró archivo .csv único en:", carpeta_temporal))
    }

    # Eliminar carpeta temporal después de mover
    unlink(carpeta_temporal, recursive = TRUE)
  }
}

In [5]:
#Listas de archivos
archivosviv<-list.files("data", full.names = TRUE, pattern = "viviendas_.*.csv")
archivospob<- list.files("data", full.names = TRUE, pattern = "poblacion_.*.csv")

archivosviv
archivospob

[1] "data/viviendas_2016.csv" "data/viviendas_2018.csv"
[3] "data/viviendas_2020.csv" "data/viviendas_2022.csv"
[5] "data/viviendas_2024.csv"

[1] "data/poblacion_2016.csv" "data/poblacion_2018.csv"
[3] "data/poblacion_2020.csv" "data/poblacion_2022.csv"
[5] "data/poblacion_2024.csv"

In [6]:
estimacion_propietarios_sexo <- function(path_viv, path_pob) {
  # Leer población y crear clave de unión
  pob <- read_csv(path_pob, show_col_types = FALSE) %>%
    mutate(claveconcat = paste0(folioviv, foliohog, numren)) %>%
    select(claveconcat, sexo)

  # Leer viviendas y crear clave de unión
  vivi <- read_csv(path_viv, show_col_types = FALSE) %>%
    mutate(claveconcat = paste0(folioviv, hog_dueno1, num_dueno1)) %>%
    #Pegar población
    left_join(pob, by = "claveconcat")

  # Crear diseño muestral
  dm <- vivi %>%
    as_survey_design(ids = upm,
                     strata = est_dis,
                     weights = factor)

  # Estimar propietarios de viviendas propias o en proceso de pago, por sexo
  vivienda_propietarios <- dm %>%
  #Filtro de viviendas propias o propias en proceso de pago
    filter(tenencia %in% c(3, 4)) %>%
    group_by(sexo) %>%
    summarise(
      viviendas = survey_total(vartype = "cv"),
      pct = survey_prop(vartype = "cv")
    ) %>%
    mutate(
      pct = round(pct * 100, 1),
      sexo = case_when(
        sexo == 1 ~ "Hombres",
        sexo == 2 ~ "Mujeres",
        TRUE ~ "No especificado"
      ),
      year = substr(path_viv, 16, 19)
    )

  return(vivienda_propietarios)
}


In [7]:
#Obtener datos para todos los años
tabla_prop_sex <- map2_dfr(
  archivosviv,
  archivospob,
  .f = ~ estimacion_propietarios_sexo(path_viv = .x, path_pob = .y)
)
tabla_prop_sex

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"One or mo

sexo,viviendas,viviendas_cv,pct,pct_cv,year
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Hombres,13794486,0.008493404,61.9,0.005681543,2016
Mujeres,8477522,0.010605616,38.1,0.009244915,2016
Hombres,14288058,0.007535759,61.9,0.004992601,2018
Mujeres,8776413,0.009718573,38.1,0.008127987,2018
Hombres,14960116,0.007438282,61.1,0.004942629,2020
Mujeres,9544340,0.008925088,38.9,0.007747240,2020
Hombres,15372727,0.007339941,59.8,0.005034697,2022
Mujeres,10340581,0.008840445,40.2,0.007484784,2022
Hombres,15635858,0.007893355,58.4,0.005486940,2024


In [8]:
#Dataviz
tabla_prop_sex%>%
select(year,sexo,viviendas,pct)%>%
#Separador de miles en viviendas
mutate(viviendas = scales::comma(viviendas, big.mark = ","))%>%
  pivot_wider(
    names_from = sexo,
    values_from = c(viviendas, pct),
    names_sep = "_"
  )%>%
    gt() %>%
      tab_options(table.font.names = 'Poppins',
              source_notes.font.size = 8) %>%
              tab_header(title = md("**Viviendas propias o en proceso de pago por sexo del dueño, 2016-2024**"),
             subtitle = md("*(Número de viviendas y porcentaje)*")) %>%
  tab_spanner(
    label = md("**Sexo**"),
    columns = c(viviendas_Hombres, viviendas_Mujeres)
  ) %>%
  tab_spanner(
    label = md("**Porcentaje**"),
    columns = c(pct_Hombres, pct_Mujeres)
  ) %>%
  cols_label(
    year= md("**Año**"),
    viviendas_Hombres = md("**Hombres**"),
    viviendas_Mujeres = md("**Mujeres**"),
    pct_Hombres = md("**Porcentaje**"),
    pct_Mujeres = md("**Porcentaje**")
  ) %>%
   cols_align(align = "center") %>%
  tab_stubhead(label = "Año")%>%
  tab_source_note(
    md("Nota: Para este cálculo se consideran las viviendas propias o en proceso de pago y el identificador del primer dueño")
  ) %>%
tab_source_note(
    md("Fuente: @claudiodanielpc con datos de INEGI. Encuesta Nacional de Ingresos y Gastos de los Hogares (ENIGH) 2024.")) %>%
  #Eliminar bordes arriba y abajo
  tab_options(table.border.top.width = px(0),
              table.border.bottom.width = px(0)) %>%
  tab_options(column_labels.background.color = "#9F2241")%>%
    #Salvar tabla
  gtsave("vivienda_prop_sex.png")